In [1]:
import pandas as pd
from pyfaidx import Fasta
from Bio.Seq import Seq

In [2]:
sha512sum = !sha512sum data/04_train_set.tsv
assert("5006dfaea420edf91d4e86a72bc2428a57ad3b184a8e3b9cc55ec048d17e001247a36322b18b48fa19b24a784b8bd7fe33eb2cfa074d4625138f4728b56d3324" in sha512sum[0])
df = pd.read_csv("data/04_train_set.tsv", sep="\t")
df

,chr,start,end,strand,class
0,chr1,12227,12612,+,1
1,chr1,12721,13220,+,1
2,chr1,12057,12178,+,1
3,chr1,12697,12974,+,1
4,chr1,13052,13220,+,1
...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0
519030,chrY,25513173,25513588,-,0
519031,chrY,25513745,25516715,-,0
519032,chrY,25525288,25527646,-,0


In [3]:
# This is downloaded in https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/
genome = Fasta('data/human_ref_hg38_109/GRCh38.primary_assembly.genome.fa', sequence_always_upper=True)

# A simple lambda function for matching the chromosome, start and end
coords_to_dna = lambda start_c, end_c, chr_c: genome[chr_c][start_c-1:end_c]

In [4]:
# def find_sequence(row):
#     chromosome = row["chr"]
#     start = row["start"]
#     stop = row["end"]
#     strand = row["strand"]
#
#     seq_find_sequence = str(coords_to_dna(int(start)+1, int(stop), chromosome))
#
#     if strand == "-": #strand
#         seq_find_sequence = Seq(seq_find_sequence)  # Encode the seqeunce into
#         seq_find_sequence = seq_find_sequence.reverse_complement()
#
#
#     return str(seq_find_sequence)

def find_sequence_maxentscan(row):
    chromosome = row["chr"]
    start = row["start"]
    stop = row["end"]
    strand = row["strand"]

    seq_find_sequence = str(coords_to_dna(int(start)+1-3, int(stop)+3, chromosome))

    if strand == "-": #strand
        seq_find_sequence = Seq(seq_find_sequence)  # Encode the sequence into
        seq_find_sequence = seq_find_sequence.reverse_complement()

    seq_find_sequence = str(seq_find_sequence)
    first_three = seq_find_sequence[:3].lower()
    last_three = seq_find_sequence[-3:].lower()
    middle_part = seq_find_sequence[3:-3].upper()

    return first_three + middle_part + last_three

In [5]:
# df_temp = df.copy().head(5000)
# df_temp["sequence"] = df_temp.apply(find_sequence, axis=1)
# df_temp["sequence_maxentscan"] = df_temp.apply(find_sequence_maxentscan, axis=1)
# df_temp

In [7]:
df["maxentscan_sequence"] = df.apply(find_sequence_maxentscan, axis=1)

In [8]:
df["start_ss"] = df["maxentscan_sequence"].str[3:5]
df["end_ss"] = df["maxentscan_sequence"].str[-5:-3:]
df

,chr,start,end,strand,class,maxentscan_sequence,start_ss,end_ss
0,chr1,12227,12612,+,1,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG
1,chr1,12721,13220,+,1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG
2,chr1,12057,12178,+,1,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA
3,chr1,12697,12974,+,1,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG
4,chr1,13052,13220,+,1,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG
...,...,...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,cacGTTGAGGCGCCCAGTGGCGGCCTCACGGGGCAGGGCGAGGGCG...,GT,AG
519030,chrY,25513173,25513588,-,0,aggGTAAGATCAGTGCTATTGTCAGAGGAAAAACTCCTGGCCATCA...,GT,AG
519031,chrY,25513745,25516715,-,0,gctGTAAGTTCCACATTGATTATCATAGGCTAACCATGGGCCAGGC...,GT,AG
519032,chrY,25525288,25527646,-,0,tgtGTGTATAAATATCTGGACTTTTTGGTTAAGTAATTATAGTTAA...,GT,AG


In [10]:
df.to_csv("data/08_trainset_with_seq.tsv", sep="\t", index=False)

In [15]:
!sha512sum data/08_trainset_with_seq.tsv

159da72ef546de9f15bfc662d51b70987134035499c64674f4240279ed9d120d34b0312bf98f1cda8c4d43222df7175f818c6d53c9dde2487ea7cf747b51ab05  data/08_trainset_with_seq.tsv
